## Notes

### Overview
This notebook captures a series of tests performed on the Raspberry Pi 2 with the Adafruit BNO055 9-dof IMU sensor board.  The main purpose of the tests was to characterize the reliability with which IMU sensor readings can be obtained for a variety of periodic sampling schedules, using a pure python implementation

### Test Design
The following tests show several performance metrics for each test case:
- Successful reads: Total successful reads (of x,y,z linear acceleration measurements)
- Wakeup wrap count: Total occurrences of a periodic wakeup that was interupted by the subsequent wakeup while waiting for the serial read operations to complete (including retries)
- Read success rate: Successful reads / Read attempts, i.e. the average success rate overall all scheduled wakeups

### Conclusions
The pure python implementation did not appear to have any higher wrap rate or comm failure rate than the C-based or Cython implementations.  This suggests that the baudrate is probably the limiting factor in data transfer, not the code latency.  (The Cython implementation is summarized here: sandboxes/ipython_notebooks/imu_sample_rate_tests_2.ipynb)

In [6]:
from __future__ import division
import time

In [43]:
from Adafruit_BNO055 import BNO055
import serial
from serial.serialutil import SerialException

class BNO055_2(BNO055.BNO055):
    def __init__(self, rst=None, address=BNO055.BNO055_ADDRESS_A, i2c=None, gpio=None,
                 serial_port=None, serial_timeout_sec=5, baudrate=115200, 
                 max_send_attempts=5, **kwargs):
        # If reset pin is provided save it and a reference to provided GPIO
        # bus (or the default system GPIO bus if none is provided).
        self._rst = rst
        if self._rst is not None:
            if gpio is None:
                import Adafruit_GPIO as GPIO
                gpio = GPIO.get_platform_gpio()
            self._gpio = gpio
            # Setup the reset pin as an output at a high level.
            self._gpio.setup(self._rst, GPIO.OUT)
            self._gpio.set_high(self._rst)
            # Wait a 650 milliseconds in case setting the reset high reset the chip.
            time.sleep(0.65)
        self._max_send_attempts=max_send_attempts
        self._serial = None
        self._i2c_device = None
        if serial_port is not None:
            # Use serial communication if serial_port name is provided.
            # Open the serial port at 115200 baud, 8N1.  Add a 5 second timeout
            # to prevent hanging if device is disconnected.
            self._serial = serial.Serial(serial_port, baudrate, timeout=serial_timeout_sec,
                                         writeTimeout=serial_timeout_sec)
        else:
            raise AttributeError('No valid serial interface specified.')
            
    def _serial_send(self, command, ack=True, max_attempts=5):
        # Send a serial command and automatically handle if it needs to be resent
        # because of a bus error.  If ack is True then an ackowledgement is
        # expected and only up to the maximum specified attempts will be made
        # to get a good acknowledgement (default is 5).  If ack is False then
        # no acknowledgement is expected (like when resetting the device).
        attempts = 0
        while True:
            # Flush any pending received data to get into a clean state.
            self._serial.flushInput()
            # Send the data.
            self._serial.write(command)
            #logger.debug('Serial send: 0x{0}'.format(binascii.hexlify(command)))
            # Stop if no acknowledgment is expected.
            if not ack:
                return
            # Read acknowledgement response (2 bytes).
            resp = bytearray(self._serial.read(2))
            #logger.debug('Serial receive: 0x{0}'.format(binascii.hexlify(resp)))
            if resp is None or len(resp) != 2:
                raise RuntimeError('Timeout waiting for serial acknowledge, is the BNO055 connected?')
            # Stop if there's no bus error (0xEE07 response) and return response bytes.
            if not (resp[0] == 0xEE and resp[1] == 0x07):
                return resp
            # Else there was a bus error so resend, as recommended in UART app
            # note at:
            #   http://ae-bst.resource.bosch.com/media/products/dokumente/bno055/BST-BNO055-AN012-00.pdf
            attempts += 1
            if attempts >=  self._max_send_attempts:
                raise RuntimeError('Exceeded maximum attempts to acknowledge serial command without bus error!')

In [22]:
def wakeup_handler(signum, frame):
    global wakeup_set, wakeup_wrap
    if wakeup_set:
        wakeup_wrap = True
    wakeup_set = True

def initialize_sensor_itimer(wakeup_interval_usec):
    import signal
    # Define sigaction for SIGALRM
    signal.signal(signal.SIGALRM,wakeup_handler)
    signal.setitimer(signal.ITIMER_REAL,wakeup_interval_usec*1e-6,wakeup_interval_usec*1e-6)
    
def cleanup_itimer():
    import signal
    signal.setitimer(signal.ITIMER_REAL,0,0)

def finalize_wakeup_attempt():
    global wakeup_set, wakeup_count, read_started, read_completed
    wakeup_set = False
    wakeup_count += 1
    read_started = False
    read_completed = True
    

In [54]:
def run_serial_read_test(num_samples, sample_interval_usec, max_read_retries):
    
    # Reset global variables (Probably necessary for repeated runs in iPython)
    global wakeup_count, wakeup_set, read_started, read_completed, wakeup_wrap
    wakeup_count = 0
    wakeup_set = 0
    read_started = 0
    read_completed = 0
    wakeup_wrap = 0
    
    # Compute operational constants
    max_time_sec = 10 * num_samples * sample_interval_usec * 1.0e-6 # Extra factor of 10, so we don't get cutoff prematurely

    
    # Create serial adapter and initialize BNO055 sensor
    imu = BNO055_2(
        serial_port='/dev/ttyAMA0', 
        rst=18, 
        baudrate=115200,
        max_send_attempts=max_read_retries)
    
    # Repeatedly Pole Sensor until end of test
    loop_count, success_count, comm_fail_count = 0,0,0
    wrap_count, max_comm_fail_count = 0,0
    duration, heartbeat_duration = 0,0
    
    # Initialize timers
    start_time = time.time()
    heartbeat_time = start_time
    
    try:
        # Initialize the interupt timer for periodic wakeups
        initialize_sensor_itimer(wakeup_interval_usec=sample_interval_usec)
        
        while True:
            loop_count+=1        
            if (loop_count % 100) == 0:
                current_time = time.time()
                duration = current_time - start_time
                if duration >= max_time_sec:
                    raise ValueError('Max program duration exceeded')
            if (loop_count > 100*1e6):
                raise ValueError('Max program loop count exceeded')
            if (wakeup_count > num_samples):
                raise ValueError('Max read attempts exceeded')
            
            # - Come in here if a scheduled wakeup occurred and has not been marked complete (by success or failure)
            if wakeup_set:
                # - Handle wakeup wrap condition
                if wakeup_wrap:
                    wakeup_wrap = False
                    wrap_count += 1
                    finalize_wakeup_attempt()
                    # In the case of wakeup wrap, keep wakeup_set true, but count the failed attempt
                    wakeup_set = True
                
                # Indicate that we have started trying to read from the sensor for the current scheduled wakeup
                read_started = 1
                read_completed = 0
        
                # DEBUG - Try to sleep for 1 sec, but expect to be interrupted
                #time.sleep(.01)
                # TODO: replace sleep with attempt to read linear accel (using max_read_retries)
                try:
                    x, y, z = imu.read_linear_acceleration()
                except RuntimeError as err:
                    print 'Runtime Exception: {}'.format(err)
                    continue
                except SerialException as err:
                    print 'Serial Exception: {}'.format(err)
                    continue
                except:
                    #print 'Unknown exception on wakeup: {}'.format(wakeup_count)
                    continue
                
                success_count += 1
                finalize_wakeup_attempt()
    
    except ValueError as err:
        print 'Program ended: {}'.format(err)
    finally:
        cleanup_itimer()
        
    # - Cleanup after main program loop
    print 'Program duration (sec): {:0.1f}'.format(duration)
    print 'Total loop count: {}'.format(loop_count)
    print 'Read attempts: {}'.format(wakeup_count)
    print 'Sensor sample interval (ms): {:0.1f}'.format(sample_interval_usec/1000);
    print 'Successful reads: {}'.format(success_count)
    #printf("Max comm failure occurrences: %d\n",max_comm_fail_count);
    print 'Wakeup wrap count: {}'.format(wrap_count);
    print 'Read success rate: {:0.3f}'.format(success_count/wakeup_count);


### Experiments

In [48]:
run_serial_read_test(num_samples=1000, sample_interval_usec=50000, max_read_retries=5)

Program ended: Max read attempts exceeded
Program duration (sec): 50.0
Total loop count: 5003210
Read attempts: 1001
Sensor sample interval (ms): 50.0
Successful reads: 1001
Wakeup wrap count: 0
Read success rate: 1.000


In [49]:
run_serial_read_test(num_samples=1000, sample_interval_usec=10000, max_read_retries=5)

Program ended: Max read attempts exceeded
Program duration (sec): 10.0
Total loop count: 963311
Read attempts: 1001
Sensor sample interval (ms): 10.0
Successful reads: 1000
Wakeup wrap count: 1
Read success rate: 0.999


In [57]:
run_serial_read_test(num_samples=1000, sample_interval_usec=7500, max_read_retries=5)

Program ended: Max read attempts exceeded
Program duration (sec): 7.5
Total loop count: 653981
Read attempts: 1001
Sensor sample interval (ms): 7.5
Successful reads: 999
Wakeup wrap count: 2
Read success rate: 0.998


In [58]:
run_serial_read_test(num_samples=1000, sample_interval_usec=5000, max_read_retries=5)

Program ended: Max read attempts exceeded
Program duration (sec): 5.0
Total loop count: 316423
Read attempts: 1001
Sensor sample interval (ms): 5.0
Successful reads: 997
Wakeup wrap count: 4
Read success rate: 0.996


In [55]:
run_serial_read_test(num_samples=1000, sample_interval_usec=1000, max_read_retries=5)

Program ended: Max read attempts exceeded
Program duration (sec): 1.2
Total loop count: 7363
Read attempts: 1001
Sensor sample interval (ms): 1.0
Successful reads: 318
Wakeup wrap count: 683
Read success rate: 0.318
